In [ ]:
import pandas as pd

# Läs in data
df = pd.read_csv("nordtech_data.csv")
df_val = pd.read_csv("nordtech_validation.csv")

# Grundläggande EDA
df.head(), df.info()


In [ ]:
import pandas as pd
import numpy as np

# Läs in rådata
df = pd.read_csv("nordtech_data.csv")
df_val = pd.read_csv("nordtech_validation.csv")

df.head(), df.info()


In [ ]:
def parse_swedish_dates(series):
    swedish_months = {
        "januari": "January",
        "februari": "February",
        "mars": "March",
        "april": "April",
        "maj": "May",
        "juni": "June",
        "juli": "July",
        "augusti": "August",
        "september": "September",
        "oktober": "October",
        "november": "November",
        "december": "December"
    }

    s = series.astype(str).str.lower()
    for swe, eng in swedish_months.items():
        s = s.str.replace(swe, eng, regex=False)

    return pd.to_datetime(s, errors="coerce")


df["orderdatum_dt"] = parse_swedish_dates(df["orderdatum"])
df["leveransdatum_dt"] = parse_swedish_dates(df["leveransdatum"])


In [ ]:
def clean_price(series):
    cleaned = (
        series.astype(str)
        .str.lower()
        .str.replace("sek", "", regex=False)
        .str.replace("kr", "", regex=False)
        .str.replace(" ", "", regex=False)
        .str.replace(",", ".", regex=False)
        .str.replace(r"[^0-9.]", "", regex=True)
    )
    return pd.to_numeric(cleaned, errors="coerce")


df["pris_enhet_sek"] = clean_price(df["pris_per_enhet"])


In [ ]:
df[["pris_per_enhet", "pris_enhet_sek"]].head(10)


In [ ]:
# Ta bort rader utan giltigt pris
df = df[df["pris_enhet_sek"].notna()]


In [ ]:
# Antal
df["antal"] = pd.to_numeric(df["antal"], errors="coerce")

# Betalmetod
df["betalmetod"] = (
    df["betalmetod"]
    .str.strip()
    .str.lower()
    .replace({
        "faktura": "Faktura",
        "kort": "Kort",
        "swish": "Swish"
    })
)

# Region
df["region"] = (
    df["region"]
    .str.lower()
    .replace({
        "stckhlm": "stockholm",
        "stocholm": "stockholm"
    })
    .str.title()
)


In [ ]:
df["region"] = df["region"].fillna("Okänd")
df["betalmetod"] = df["betalmetod"].fillna("Okänd")
df["leveransstatus"] = df["leveransstatus"].fillna("Okänd")

# Filtrera bort orimliga datum
df = df[df["leveransdatum_dt"] >= df["orderdatum_dt"]]


In [ ]:
df["rad_total"] = df["antal"] * df["pris_enhet_sek"]

df["leveranstid_dagar"] = (
    df["leveransdatum_dt"] - df["orderdatum_dt"]
).dt.days

df["vecka"] = df["orderdatum_dt"].dt.isocalendar().week
df["månad"] = df["orderdatum_dt"].dt.month

df.head()


### Sentimentanalys

Kundrecensioner analyserades med en förtränad BERT-modell.
Resultatet klassificerades som positivt, neutralt eller negativt.


In [ ]:
from transformers import pipeline

sentiment_model = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment",
    framework="pt"
)


In [ ]:
def classify_sentiment(text):
    if pd.isna(text):
        return "Ingen recension"
    
    try:
        result = sentiment_model(text[:512], truncation=True)[0]
        stars = int(result["label"][0])

        if stars <= 2:
            return "Negativ"
        elif stars == 3:
            return "Neutral"
        else:
            return "Positiv"
    except Exception:
        return "Okänt"


In [ ]:
mask = df["recension_text"].notna()
df.loc[mask, "sentiment"] = df.loc[mask, "recension_text"].apply(classify_sentiment)
df["sentiment"] = df["sentiment"].fillna("Ingen recension")


In [ ]:
df["sentiment"].value_counts()


**Resultat av sentimentanalys**

Majoriteten av orderraderna saknar kundrecension, vilket är vanligt inom e-handel.
Bland inkomna recensioner dominerar positiva omdömen, följt av neutrala och negativa.
Detta indikerar överlag god kundnöjdhet, samtidigt som negativa recensioner pekar på områden med förbättringspotential.


In [ ]:
df[["recension_text", "betyg", "sentiment"]].dropna().head(10)


In [ ]:
import sqlite3

### Load till SQLite

Den rensade och transformerade datan sparades i en lokal SQLite-databas.
Efter laddning verifierades datans innehåll genom återläsning och radantal.


In [ ]:
# Skapa (eller anslut till) SQLite-databas
conn = sqlite3.connect("nordtech.db")

# Spara dataframe till databas
df.to_sql(
    name="orders_clean",
    con=conn,
    if_exists="replace",
    index=False
)

conn.close()

In [ ]:
conn = sqlite3.connect("nordtech.db")

row_count = pd.read_sql(
    "SELECT COUNT(*) AS rows FROM orders_clean",
    conn
)

conn.close()

row_count


In [ ]:
df.shape

## KPI 1 – Försäljning per vecka

Denna KPI visar hur den totala försäljningen utvecklas över tid,
vilket kan användas för att identifiera säsongsmönster och perioder
med hög eller låg efterfrågan.


In [ ]:
weekly_sales = (
    df.groupby("vecka")["rad_total"]
    .sum()
    .reset_index()
    .sort_values("vecka")
)

weekly_sales.head()


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(weekly_sales["vecka"], weekly_sales["rad_total"])
plt.xlabel("Vecka")
plt.ylabel("Total försäljning (SEK)")
plt.title("Total försäljning per vecka")
plt.show()


**Tolkning:**  
Försäljningen varierar mellan veckor med tydliga toppar, vilket kan
indikera kampanjer eller säsongsvariationer.


## KPI 2 – Genomsnittligt ordervärde (AOV)

Det genomsnittliga ordervärdet visar hur mycket en kund i snitt
spenderar per order. KPI:n är central för att förstå kundbeteende
och lönsamhet, särskilt inom e-handel.


In [ ]:
order_values = (
    df.groupby("order_id")["rad_total"]
    .sum()
)

order_values.head()


In [ ]:
# Exkludera ordrar med 0 kr
order_values_nonzero = order_values[order_values > 0]

# Beräkna genomsnittligt ordervärde
average_order_value = order_values_nonzero.mean()
average_order_value


**Tolkning:**  
Det genomsnittliga ordervärdet beräknades efter att ordrar med nollvärde
exkluderats. Dessa bedöms inte representera faktiska försäljningstillfällen
och skulle annars snedvrida KPI:n. Resultatet ger därför en mer rättvisande
bild av kundernas genomsnittliga köpbeteende.



## KPI 3 – Leveranstid

Leveranstid mäter antalet dagar mellan orderdatum och leveransdatum.
KPI:n är central för kundupplevelsen, då längre leveranstider ofta
påverkar kundnöjdheten negativt.


In [ ]:
df["leveranstid_dagar"].describe()


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.hist(df["leveranstid_dagar"].dropna(), bins=20)
plt.xlabel("Leveranstid (dagar)")
plt.ylabel("Antal orderrader")
plt.title("Fördelning av leveranstid")
plt.show()


**Tolkning:**  
De flesta leveranser sker inom ett begränsat antal dagar, vilket indikerar
en relativt effektiv logistik. Samtidigt förekommer längre leveranstider,
vilket kan påverka kundupplevelsen negativt och bör analyseras vidare.


## KPI 4 – Kundnöjdhet per produktkategori

Denna KPI analyserar kundnöjdhet baserat på sentiment från
kundrecensioner, uppdelat per produktkategori. Syftet är att identifiera
kategorier med hög respektive låg kundnöjdhet.


In [ ]:
sentiment_by_category = (
    df[df["sentiment"] != "Ingen recension"]
    .groupby(["kategori", "sentiment"])
    .size()
    .unstack(fill_value=0)
)

sentiment_by_category


In [ ]:
sentiment_by_category.plot(kind="bar", stacked=True)
plt.xlabel("Produktkategori")
plt.ylabel("Antal recensioner")
plt.title("Kundnöjdhet per produktkategori")
plt.show()


**Tolkning:**  
Analysen visar att kundnöjdheten varierar mellan produktkategorier.
Vissa kategorier domineras av positiva recensioner, medan andra har en
högre andel neutrala eller negativa omdömen. Detta kan indikera skillnader
i produktkvalitet, leveransförväntningar eller kundservice kopplat till
specifika kategorier.
